In [39]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.core import Model

from azureml.exceptions import ComputeTargetException
from azureml.widgets import RunDetails

In [41]:
# important parameters
compute_name = 'dp-100-v2'
project_folder = '../'

# 1. Configure Workspace

In [42]:
ws = Workspace.from_config('../config/config.json')

# 2. Configure Compute Target

In [43]:
try:
    comput_target = ComputeTarget(workspace=ws, name= compute_name)
    print('Comput target exists!')
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_VS2'
                                                          ,vm_priority='lowpriority'
                                                          ,min_nodes=0
                                                          , max_nodes=1)
    comput_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)                                       

# 3. Configurate Dependencies

In [44]:
env = Environment.from_conda_specification(name = 'sklearn-env', file_path = '../config/conda_dependencies.yml')

# 5. Configure Experiment

In [74]:
src = ScriptRunConfig(source_directory=project_folder
                     ,script='./src/train.py'
                     ,compute_target=comput_target
                     ,arguments=['--param_1', 0.6, 
                                 '--remote_execution', True,
                                 '--path_data', './data/diabetes.csv']
                     ,environment=env)

In [75]:
experiment = Experiment(workspace=ws, name='test_v4')

# 6. Run Experiment und Monitor Training

In [76]:
run = experiment.submit(src)

In [77]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

# 7. Add trained motel to workspace

In [ ]:
model = Model.register(workspace=ws
                       ,model_path='../outputs/model.pkl'
                       ,model_name='Diabetes Classifier'
                       ,description='First trained model with Azure ML'
                       ,tags={'data-format': 'csv'}
                       ,model_framework=Model.Framework.SCIKITLEARN
                       ,model_framework_version='0.23.2')